In [ ]:
import sys
from datetime import datetime
import math

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

from pyspark.sql.types import *

In [ ]:
spark = SparkSession\
    .builder\
    .appName("SparkETL")\
    .getOrCreate()

In [ ]:
def getAngDist1(x):
    ra1 = x.SourceRA
    ra2 = x.DestRA
    dec1 = x.SourceDec
    dec2 = x.DestDec
    sin_dec1 = math.sin(math.radians(dec1))
    sin_dec2 = math.sin(math.radians(dec2))
    cos_dec1 = math.cos(math.radians(dec1))
    cos_dec2 = math.cos(math.radians(dec2))
    raDiff = math.radians((ra1 - ra2)*15)
    cos_ra = math.cos(raDiff)
    cosAng = sin_dec1*sin_dec2 + cos_dec1*cos_dec2*cos_ra
    #Note: divide by 0.01745329252 below to convert radians to degrees
    # Note2: for some reason, math.acos(0) gives a math domain error.
    # In pyspark this causes the job to crash with a confusing "unable to overwrite file" error
    if ((ra1 == ra2) & (dec1 == dec2)):
        AngDist = 0
    else:
        AngDist = math.acos(cosAng)/0.01745329252
    return(x.Source, x.Destination, AngDist, ra1, dec1, ra2, dec2)

In [ ]:
def toCSVLine(data):
  return ','.join(str(d) for d in data)

In [ ]:
#df = spark.read.csv("s3://emr-astronomy-petproj/pairsv3/pairsv2-last.csv", header=True, inferSchema=True)
df = spark.read.csv("s3://BUCKET/FOLDER/*.csv", header=True, inferSchema=True)

In [ ]:
# just for debugging - comment in final
df.count()

In [ ]:
# just for debugging - comment in final
#df.rdd.countApprox(timeout = 1000, confidence = 0.95)

In [ ]:
df.printSchema()

In [ ]:
rdd2 = df.rdd.map(lambda x: getAngDist1(x))

In [ ]:
#Note: this will result in an out of memory error for large datasets. Hence, looping through files
#rdd2.collect()

In [ ]:
#I am doing it this way since I need to get an output in a .csv readable by Athena

lines = rdd2.map(toCSVLine)
lines.saveAsTextFile("s3://BUCKET/FOLDER/OUT.csv")